### 실습 | 초음파 광물 예측하기 : 학습셋과 테스트셋 구분

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd

In [11]:
df = pd.read_csv('./data/sonar3.csv', header=None)

X = df.iloc[:, 0:60]
y = df.iloc[:, 60]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

model = Sequential()

model.add(Dense(24, activation='relu', input_dim=60))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(X_train, y_train, epochs=200, batch_size=10)

score = model.evaluate(X_test, y_test)
print('Test accuracy : {}'.format(score[1]))

Epoch 1/200
15/15 [==============================] - 1s 5ms/step - loss: 0.6880 - acc: 0.5103
Epoch 2/200
15/15 [==============================] - 0s 4ms/step - loss: 0.6722 - acc: 0.5586
Epoch 3/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6664 - acc: 0.5862
Epoch 4/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6597 - acc: 0.5931
Epoch 5/200
15/15 [==============================] - 0s 3ms/step - loss: 0.6527 - acc: 0.5862
Epoch 6/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6421 - acc: 0.6138
Epoch 7/200
15/15 [==============================] - 0s 3ms/step - loss: 0.6299 - acc: 0.6621
Epoch 8/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6171 - acc: 0.7103
Epoch 9/200
15/15 [==============================] - 0s 3ms/step - loss: 0.6041 - acc: 0.7172
Epoch 10/200
15/15 [==============================] - 0s 3ms/step - loss: 0.5892 - acc: 0.7034
Epoch 11/200
15/15 [==============================] - 0s 2m

In [12]:
# 모델 이름과 저장할 위치를 함께 지정
model.save('./data/model/my_model.keras')

In [13]:
# 모델이 저장된 위치와 이름까지 적어 준다.
from keras.models import Sequential, load_model

model = load_model('./data/model/my_model.keras')

In [14]:
score = model.evaluate(X_test, y_test)
print(f'Test accuracy : {score[1]}')

2/2 [==============================] - 1s 4ms/step - loss: 0.6081 - acc: 0.8571
Test accuracy : 0.8571428656578064


### 실습 | 초음파 광물 예측하기 : k겹 교차검증

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import pandas as pd

In [16]:
df = pd.read_csv('./data/sonar3.csv', header=None)

X = df.iloc[:, 0:60]
y = df.iloc[:, 60]

k = 5

kfold = KFold(n_splits=k, shuffle=True)
acc_score = []

def model_fn():
    model = Sequential()
    model.add(Dense(24, activation='relu', input_dim=60))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model = model_fn()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X_train, y_train, epochs=200, batch_size=10, verbose=0)
    
    accuracy = model.evaluate(X_test, y_test)[1]
    acc_score.append(accuracy)

avg_acc_score = sum(acc_score) / k

print('정확도 : {}'.format(acc_score))
print('정확도 평균 : {}'.format(avg_acc_score))

2/2 [==============================] - 0s 5ms/step - loss: 1.0465 - acc: 0.8293
정확도 : [0.8333333134651184, 0.8571428656578064, 0.7857142686843872, 0.8780487775802612, 0.8292682766914368]
정확도 평균 : 0.836701500415802
